In [10]:
import timeit
import numpy as np
from time import time

In [11]:
from numba import cuda
import numba as nb
import math

In [12]:
def init_kernel_bias(num_inp_channels, kernel_size, num_kernels,mean=0,std=0.01):
    shape = [num_inp_channels, kernel_size, kernel_size, num_kernels]
    weights = std*np.random.randn(*shape) + mean
    # weights/=np.sqrt(num_inp_channels)
    bias = std*np.random.randn(1,num_kernels) + mean
    return weights, bias

In [13]:
w0,b0=init_kernel_bias(num_inp_channels=32,kernel_size=3,num_kernels=128)

In [14]:
inp=np.random.randn(128,64,64,32)

In [15]:
#inp[batches,row,col,d],w0(d,ksz,ksz,num_ker),b0[1,num_ker],stride[row,col]
padding=0
stride=[1,1]
ipp=inp.transpose(0,3,1,2)  #ipp[batches,d,row,col]
output=[]
ksz=w0.shape[1]
num_ker=w0.shape[3]
if not padding: #take care of padding in backprop too
    padding=(ksz-1)//2  #currently don't give 'even' ksz
out_row,out_col=((ipp.shape[2]-ksz+2*padding)//stride[0]+1),((ipp.shape[3]-ksz+2*padding)//stride[1]+1)
batches,d,row,col=ipp.shape
row+=2*padding
col+=2*padding
padded=np.zeros((batches,d,row,col))
padded[:,:,padding:-padding,padding:-padding]=ipp

In [16]:
img=padded[0]

In [17]:
# %%timeit
window=(np.arange(ksz)[:,None]*row+np.arange(ksz)).ravel()+np.arange(d)[:,None]*row*col
slider=(np.arange(out_row*stride[0])[:,None]*row+np.arange(out_col*stride[1]))
ind = window.ravel()+slider[::stride[0],::stride[1]].ravel()[:,None]
# bind= np.arange(batches)[:,None]*d*row*col+ind.ravel()
kern = w0.reshape(-1,num_ker)
# output=(np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)).reshape(batches,out_row,out_col,num_ker)

In [18]:
print(cuda.gpus)

<Managed Device 0>


In [19]:
cuda.select_device(0)

<weakproxy at 0x7f50917a3c28 to Device at 0x7f503f854a90>

In [42]:
# %%time
# Initialize the data arrays
A = np.asfortranarray(np.random.randn(32*2, 32*3))
B = np.asfortranarray(np.random.randn(3*32, 32))

In [50]:
%%time
# Copy the arrays to the device
A_global_mem = cuda.to_device(A)
B_global_mem = cuda.to_device(B)

# Allocate memory on the device for the result
C_global_mem = cuda.device_array((32*2, 32))

CPU times: user 4.7 ms, sys: 4.09 ms, total: 8.79 ms
Wall time: 7.51 ms


In [7]:
%%time
# Configure the blocks
threadsperblock = (16,16)
blockspergrid_x = int(math.ceil(A.shape[0] / threadsperblock[0]))
blockspergrid_y = int(math.ceil(B.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

CPU times: user 10 µs, sys: 1 µs, total: 11 µs
Wall time: 13.4 µs


In [8]:
blockspergrid

(4, 2)

In [9]:
(A.nbytes+B.nbytes+np.dot(A,B).nbytes)/1024/1024

0.0859375

In [44]:
# CUDA kernel
@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp

In [45]:
%%time
matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)

CPU times: user 189 ms, sys: 272 µs, total: 190 ms
Wall time: 188 ms


In [46]:
%%time
C = C_global_mem.copy_to_host()
print(C.shape)

(64, 32)
CPU times: user 710 µs, sys: 66 µs, total: 776 µs
Wall time: 607 µs


In [54]:
%%time
np.allclose(C,np.dot(A,B))

CPU times: user 762 µs, sys: 0 ns, total: 762 µs
Wall time: 515 µs


False

In [48]:
np.dot(A,B)

array([[ -0.21309449,   2.89234672,  28.20324537, ..., -14.19437499,
          3.97631953, -19.85510025],
       [  7.79791636,   8.69099402, -19.05600696, ...,   5.10313222,
         23.15987213, -14.31547735],
       [ -5.62363989, -11.57985623,  -0.81119961, ...,   6.15165425,
          6.46526696, -15.27875407],
       ...,
       [-15.96582059,  -1.89555769,   4.86732631, ...,  17.79446482,
          4.05454805,  20.05742225],
       [  6.26804982,  15.04490977,   0.58337399, ...,  -8.24276073,
        -22.72991009, -23.79241361],
       [  5.1795037 ,  -9.69664089,  -9.41470239, ...,   3.58212059,
          0.82304345,   5.880077  ]])

In [53]:
C

array([[ -0.21309448,   2.89234644,  28.20324633, ..., -14.19437491,
          3.97631955, -19.85510151],
       [  7.79791662,   8.69099371, -19.05600668, ...,   5.10313206,
         23.15987269, -14.31547733],
       [ -5.62363966, -11.57985586,  -0.81119891, ...,   6.15165372,
          6.46526699, -15.27875432],
       ...,
       [-15.96581962,  -1.89555792,   4.86732652, ...,  17.79446452,
          4.05454818,  20.0574224 ],
       [  6.2680493 ,  15.04490923,   0.58337351, ...,  -8.24276069,
        -22.72991012, -23.79241307],
       [  5.17950452,  -9.69664087,  -9.41470208, ...,   3.58212069,
          0.82304406,   5.88007726]])

In [101]:
%time
@cuda.jit
def my_kernel_2D(io_array):
    x, y = cuda.grid(2)
    if x<io_array.shape[0] and y<io_array.shape[1]:
        io_array[x,y]*=8

data = np.ones((16, 16))
data_glob=cuda.to_device(data)
threadsperblock = (32, 32)
blockspergrid_x = math.ceil(data.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(data.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)
my_kernel_2D[blockspergrid, threadsperblock](data_glob)
print(data_glob.copy_to_host())

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs
[[8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]
 [8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]
 [8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]
 [8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]
 [8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]
 [8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]
 [8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]
 [8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]
 [8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]
 [8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]
 [8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]
 [8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]
 [8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]
 [8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]
 [8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]
 [8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]]


In [102]:
%%time
output=np.empty((batches,out_row*out_col,num_ker))
for i,img in enumerate(padded):      #img[d,row,col]
    # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
    output[i]=np.dot(img.take(ind), kern)
output+=b0
ans2=output.reshape(batches,out_row,out_col,num_ker)

CPU times: user 4.79 s, sys: 163 ms, total: 4.96 s
Wall time: 1.33 s


In [19]:
A.shape,B.shape

((64, 96), (96, 32))

In [50]:
TPB=16
@cuda.jit
def fast_matmul(A,B,C):
    sA=cuda.shared.array(shape=(TPB,TPB),dtype=nb.float32)
    sB=cuda.shared.array(shape=(TPB,TPB),dtype=nb.float32)
    x,y=cuda.grid(2)
    tx=cuda.threadIdx.x
    ty=cuda.threadIdx.y
    if x>=C.shape[0] and y>=C.shape[1]:
        return
    tmp=0
    for i in range(A.shape[1]//TPB):  # for number of blocks
        sA[tx,ty]=A[x,ty+i*TPB]       # preLoad data into shared memory
        sB[tx,ty]=B[tx+i*TPB,y]
        
        cuda.syncthreads()            # wait for loading
        for j in range(TPB):
            tmp+=sA[tx,j]*sB[j,ty]
            cuda.syncthreads()            # wait for computation
    C[x,y]=tmp

In [51]:
%%time
fast_matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)

CPU times: user 252 ms, sys: 13 ms, total: 265 ms
Wall time: 269 ms


In [52]:
%%time
C = C_global_mem.copy_to_host()
print(C.shape)

(64, 32)
CPU times: user 358 µs, sys: 269 µs, total: 627 µs
Wall time: 453 µs


In [33]:
import pycuda

In [1]:
import ctypes

In [2]:
cudlib=ctypes.CDLL('libcublas.so')

In [46]:
N=128

In [47]:
A = np.array(np.arange(N ** 2, dtype=np.float32).reshape(N, N), order='F')
B = np.array(np.arange(N) + 10, dtype=A.dtype, order='F')
D = np.zeros_like(A, order='F')

In [56]:
cudlib.cublasZgemm('N', 'N', N, N, N, 1, A_global_mem, np.diag(B), 1.0, D)

ArgumentError: argument 7: <class 'TypeError'>: Don't know how to convert parameter 7

In [32]:
from timeit import default_timer as timer

import numpy as np

from accelerate.cuda.blas import Blas


N = 128     # no. of rows/cols


def gemm_v1():
    '''
    Note that all arrays are in Fortran order.
    '''
    print("Version 1".center(80, '='))
    # Prepare arrays for input
    A = np.array(np.arange(N ** 2, dtype=np.float32).reshape(N, N), order='F')
    B = np.array(np.arange(N) + 10, dtype=A.dtype, order='F')
    D = np.zeros_like(A, order='F')

    # NumPy
    start = timer()
    E = np.dot(A, np.diag(B))
    numpy_time = timer() - start
    print("Numpy took %f seconds" % numpy_time)

    # cuBLAS
    blas = Blas()

    start = timer()
    blas.gemm('N', 'N', N, N, N, 1.0, A, np.diag(B), 1.0, D)
    cuda_time = timer() - start

    print("CUBLAS took %f seconds" % cuda_time)
    diff = np.abs(D - E)
    print("Maximum error %f" % np.max(diff))


def gemm_v2():
    """
    Let GEMM transpose the input matrices so that they can be in C order,
    originally.  Note that the output matrix is still in Fortran array.
    The string arguments in gemm tells it to apply transformation on the input
    matrices.
    See argument description in:
        http://docs.continuum.io/accelerate/cublas#blas-level-2
    """
    print("Version 2".center(80, '='))
    # Prepare arrays for input
    A = np.array(np.arange(N ** 2, dtype=np.float32).reshape(N, N))
    B = np.array(np.arange(N) + 10, dtype=A.dtype)
    D = np.zeros_like(A, order='F')

    # NumPy
    start = timer()
    E = np.dot(A, np.diag(B))
    numpy_time = timer() - start
    print("Numpy took %f seconds" % numpy_time)

    # cuBLAS
    blas = Blas()

    start = timer()
    blas.gemm('T', 'T', N, N, N, 1.0, A, np.diag(B), 1.0, D)
    cuda_time = timer() - start

    print("CUBLAS took %f seconds" % cuda_time)
    diff = np.abs(D - E)
    print("Maximum error %f" % np.max(diff))


def main():
    gemm_v1()
    gemm_v2()


if __name__ == '__main__':
   main()

ModuleNotFoundError: No module named 'accelerate'